# Instruções para o uso do Notebook

Para rodar o notebook a seguir corretamente, é necessário seguir alguns passos prévios. Primeiro, baixe os datasets disponíveis no link a seguir: [Datasets](https://drive.google.com/drive/folders/1bm6iQenyZ63gbw_s7j0md8UaHP84O0Vn?usp=sharing) (não é necessário todos, a princípio, somente os citados abaixo). Esses datasets não estão incluídos no notebook devido ao seu tamanho, por isso são ignorados pelo arquivo `.gitignore`. Todos eles devem ser colocados na pasta [`data`](data), localizada em `src/Notebook/Word2Vec_Sprint_3/data`.

Os arquivos necessários são:
- **GoogleNews-vectors-negative300.bin**: [Download](https://drive.google.com/open?id=1IzDDngIEWzGP4onWlBCglAQ87g5gP2CL&usp=drive_copy) - contém palavras pré-treinadas em 300 dimensões.
- **processed_text_data.csv**: [Download](https://drive.google.com/file/d/1hCW3pliSKDWKyr0h0tvNBtUgtw9EsE3c/view?usp=sharing) - contém o banco de dados processado (a etapa de pré-processamento pode ser acompanhado no arquivo [BOW.ipynb](../BoW_Sprint_2/BOW.ipynb)).

Todos são necessários para rodar os modelos de Word2Vec.

Outras recomendações:

- Mantenha o C++ do seu computador atualizado, pois determinadas bibliotecas são compiladas com essa linguagem.
- Caso queira conferir as versões mais atualizaas das bibliotecas, elas estão detalhadas no aquivo [``requirements.txt``](../../requirements.txt). Para usar este arquivo, basta rodar no terminal o comando: ``pip install -r requirements.txt``. Porém, a princípio, é possível ter acesso a todas as bilioteca ao rodar a célula de "Instalação e Importação das Bibliotecas" deste Notebook.

# Instalação e Importação das Bibliotecas

Na seção "Instalação e Importação das Bibliotecas", prepara-se o ambiente de codificação configurando e carregando as bibliotecas necessárias para o projeto. Esta seção é importante pelos seguintes motivos:

1. **Instalação de Bibliotecas**: Utilizam-se comandos como `%pip install nome_da_biblioteca` para instalar bibliotecas Python que não estão presentes por padrão no ambiente de execução, mas que são essenciais para a execução do código. Este comando mágico oferece uma integração mais direta com o IPython, garantindo que as instalações ocorram no kernel correto do Python utilizado pelo Jupyter.

2. **Importação de Bibliotecas**: Após a instalação, as bibliotecas são importadas usando o comando `import`, permitindo o acesso às funções e ferramentas disponibilizadas por elas. Comandos típicos incluem `import numpy as np` e `import pandas as pd`. Esta prática garante que todas as funcionalidades necessárias estejam disponíveis e prontas para uso nas seções subsequentes do notebook.

Esta seção é posicionada no início do notebook para assegurar que todas as dependências estejam corretamente configuradas antes de prosseguir com a análise de dados ou modelagem.

In [ ]:
# Instalação de "Bibliotecas Mágicas"
%pip install pandas
%pip install numpy
%pip install nltk
%pip install emoji
%pip install wordcloud
%pip install emot
%pip install imbalanced-learn
%pip install matplotlib
%pip install scikit-learn
%pip install spacy
!python -m spacy download en_core_web_md
%pip install scipy==1.11
%pip install gensim
%pip install scikit-learn
%pip install seaborn
%pip install xgboost

In [ ]:
# Importação de Bibliotecas
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import accuracy_score, confusion_matrix, roc_curve, roc_auc_score
import tempfile
import os
from gensim.models import KeyedVectors
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier
import seaborn as sns
from sklearn.svm import SVC
from sklearn.preprocessing import label_binarize
from sklearn.preprocessing import StandardScaler
from itertools import cycle
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import OneHotEncoder
from keras.models import Sequential
from keras.layers import LSTM, Dense

# Word2Vec
Word2Vec é uma técnica de aprendizado de máquina que transforma palavras em vetores numéricos, onde palavras com significados semelhantes têm representações próximas. Utilizando modelos de redes neurais, captura a semântica das palavras, sendo crucial para tarefas de processamento de linguagem natural, como análise de sentimentos, tradução automática e chatbots, devido à sua eficiência e capacidade de melhorar a compreensão contextual em grandes corpora de texto.

Essa seção contém:
- **Word2Vec de 300 dimensões**: Embeddings de palavras representadas por vetores de 300 dimensões, proporcionando uma representação mais detalhada das palavras.
- **Análise das métricas do Modelo Word2Vec de 300 dimensões**: Avaliação do desempenho do modelo de 300 dimensões.

No projeto em questão, o dataset de treinamento está em inglês, justificando a escolha do modelo `GoogleNews-vectors-negative300.bin` para garantir uma melhor adequação e desempenho nas tarefas de PLN específicas. Além disso, maior dimensionalidade permite que o modelo capture mais nuances semânticas e relacionais entre as palavras, tendem a se sair melhor em tarefas complexas de PLN e podem generalizar melhor e representar com mais precisão as relações entre palavras.

A documentação dessa seção, com suas análises descritiva e crítica, pode ser acessada [nesse link](https://github.com/Inteli-College/2024-1B-T10-SI06-G03/blob/main/docs/documentacao.md#c7).

## Word2Vec 300 dimensões 
Essa seção contém o uso de Word2Vec de 300 dimensões, utilizando o conjunto de vetores pré-treinados GoogleNews em inglês, especificamente o arquivo `GoogleNews-vectors-negative300.bin`, que pode ser acessado [nesse link](https://drive.google.com/file/d/1IzDDngIEWzGP4onWlBCglAQ87g5gP2CL/view?usp=drive_link). As subseções incluem:

- **Vetores de Palavras Pré-Treinadas 300 dimensões**: Carregamento e preparação dos vetores de palavras com 300 dimensões pré-treinados em inglês.
- **Modelo Word2Vec em 300 dimensões**: Implementação do modelo Word2Vec utilizando os vetores GoogleNews de 300 dimensões.
- **Teste isolado com conjunto de dados fictício**: Avaliação do modelo com um conjunto de dados fictício para validar a eficácia dos vetores de 300 dimensões.

Orientação: para rodar esse modelo, deve-se baixar o aquivo `GoogleNews-vectors-negative300.bin` no link oferecido e colocá-lo na pasta [data](data).

### Vetores de Palavras Pré-Treinadas 300 dimensões

In [ ]:
EMBEDDING_FILE = './data/GoogleNews-vectors-negative300.bin'
word_vectors = KeyedVectors.load_word2vec_format(EMBEDDING_FILE, binary=True)

### Modelo Word2Vec 300 dimensões

In [ ]:
def prepare_word2vec_vectors(csv_file_path, embedding_file):
    """
    Carrega dados de um arquivo CSV e utiliza um modelo Word2Vec pré-treinado para
    gerar representações vetoriais dos textos.

    Inputs:
        csv_file_path (str): Caminho para o arquivo CSV contendo a coluna de texto.
        embedding_file (str): Caminho para o arquivo binário do modelo Word2Vec pré-treinado.

    Output: DataFrame: DataFrame com os vetores de palavras para cada documento.

    """
    # Carregar o modelo pré-treinado
    word_vectors = KeyedVectors.load_word2vec_format(embedding_file, binary=True)

    # Ler o arquivo CSV
    data = pd.read_csv(csv_file_path)

    # Assume-se que a coluna com texto processado é chamada 'processed_text'
    texts = data['processed_text'].astype(str)

    # Inicializar um vetorizador para tokenizar os textos
    vectorizer = CountVectorizer(token_pattern=r'\b\w+\b')  # Tokeniza palavras
    vectorizer.fit(texts)

    # Inicializar matriz para armazenar vetores de palavras
    word_embeddings = np.zeros((len(texts), word_vectors.vector_size))

    for i, text in enumerate(texts):
        tokens = vectorizer.build_tokenizer()(text)
        word_vecs = [word_vectors[word] for word in tokens if word in word_vectors.key_to_index]
        if word_vecs:
            word_embeddings[i] = np.sum(word_vecs, axis=0)  

    # Criar um DataFrame com os vetores
    return pd.DataFrame(word_embeddings)

In [ ]:
# Chamada da Função
embedding_file = './data/GoogleNews-vectors-negative300.bin'
csv_file_path = './data/processed_text_data.csv'
vector_df = prepare_word2vec_vectors(csv_file_path, embedding_file)

In [ ]:
# Transforma em arquivo csv
vector_df.to_csv('./data/word_2_vec_text_data_300.csv', index=False)
vector_df.head()

### Teste isolado com conjunto de dados fictício

O teste abaixo verifica se o tamanho da matriz está correta.

In [ ]:
data_text_dict = {
'processed_text': [
    'uber is cool',
    'i do not like uber',
    'data science'
]
}
data_text = pd.DataFrame(data_dict)

# Criar um arquivo CSV temporário
with tempfile.NamedTemporaryFile(delete=False, suffix='.csv') as temp_csv:
    csv_input_path = temp_csv.name
    data_text.to_csv(csv_input_path, index=False)


embedding_file = './data/GoogleNews-vectors-negative300.bin'
csv_file_path = csv_input_path
vector_df = prepare_word2vec_vectors(csv_file_path, embedding_file)

# Printar somente a saída de vetor no final
print(vector_df)

# Remover o arquivo temporário
os.remove(csv_input_path)

# Deveria retornar um arquivos de 3 linhas e 300 colunas 

O teste abaixo pretende verificar se o vetor da frase completa é igual à soma dos vetores das palavras individuais da frase.

In [ ]:
v_uber = word_vectors['uber']
v_uber[0]

In [ ]:
v_is = word_vectors['is']
v_is[0]

In [ ]:
v_good = word_vectors['good']
v_good[0]

In [ ]:
v_test = v_uber + v_is + v_good
v_test[0]

In [ ]:
sentence = 'uber is good'

# Criar um arquivo CSV temporário com a frase completa
with tempfile.NamedTemporaryFile(delete=False, suffix='.csv') as temp_csv_sentence:
    csv_sentence_path = temp_csv_sentence.name
    pd.DataFrame({'processed_text': [sentence]}).to_csv(csv_sentence_path, index=False)


embedding_file = './data/GoogleNews-vectors-negative300.bin'

# Obter o vetor da frase completa usando a função
vector_df_sentence = prepare_word2vec_vectors(csv_sentence_path, embedding_file)
full_text_vector = vector_df_sentence.iloc[0].values

# Visualizar a matriz da frase completa (primeiras colunas)
print("Vetor da frase completa 'uber is good':")

print(full_text_vector[0])

# Remover os arquivos temporários
os.remove(csv_sentence_path)

## Análise do Word2Vec de 300 dimensões

### Análise de Clusters
- Análise de Agrupamento (Clustering) com K-means
- Visualização dos Vetores com t-SNE
- Distribuição dos vetores de palavras entre os diferentes clusters.

Ambos os métodos de visualização complementam-se e fornecem uma compreensão mais profunda da estrutura e dos relacionamentos entre os vetores de palavras. A aplicação desses métodos pode ser útil para várias tarefas de NLP, como melhoria de modelos, análise semântica e exploração de dados.

A descrição desses gráficos, bem como sua análise, podem ser acessadas na seção 7.2.1 Análise Gráfica do Modelo Word2Vec da documentação oficial do projeto, disponível [nesse link](https://github.com/Inteli-College/2024-1B-T10-SI06-G03/blob/main/docs/documentacao.md#c7.2.1).

In [ ]:
import pandas as pd
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt

In [ ]:
# Carregar o arquivo CSV
file_path = './data/word_2_vec_text_data_300.csv'
data = pd.read_csv(file_path)

# Definir o número de clusters
num_clusters = 5
kmeans = KMeans(n_clusters=num_clusters, random_state=0).fit(data)

# Adicionar os rótulos dos clusters ao DataFrame
data['cluster'] = kmeans.labels_

# Reduzir a dimensionalidade para visualização com PCA
pca = PCA(n_components=2)
reduced_data = pca.fit_transform(data.iloc[:, :-1])

# Visualizar os clusters (PCA)
plt.figure(figsize=(10, 8))
for i in range(num_clusters):
    points = reduced_data[data['cluster'] == i]
    plt.scatter(points[:, 0], points[:, 1], label=f'Cluster {i}')
plt.title("Clusters de Vetores de Palavras (PCA)")
plt.xlabel("PCA 1")
plt.ylabel("PCA 2")
plt.legend()
plt.show()

In [ ]:
# Reduzir a dimensionalidade para 2D usando t-SNE
tsne = TSNE(n_components=2, random_state=0)
tsne_results = tsne.fit_transform(data.iloc[:, :-1])

# Visualizar os vetores de palavras (t-SNE)
plt.figure(figsize=(10, 8))
plt.scatter(tsne_results[:, 0], tsne_results[:, 1], c=data['cluster'], cmap='viridis')
plt.title("Visualização t-SNE dos Vetores de Palavras")
plt.xlabel("t-SNE 1")
plt.ylabel("t-SNE 2")
plt.colorbar()
plt.show()

In [ ]:
# Contar o número de vetores em cada cluster
cluster_counts = data['cluster'].value_counts()

# Plotar o histograma da distribuição dos clusters
plt.figure(figsize=(10, 6))
cluster_counts.plot(kind='bar')
plt.title("Distribuição dos Vetores de Palavras entre os Clusters")
plt.xlabel("Cluster")
plt.ylabel("Número de Vetores de Palavras")
plt.show()

# Treinamento dos Modelos

A seguir, apresenta-se o código da função para a vetorização do banco de dados utilizando Word2Vec (chamada da função) e a criação do dataset para o treinamento dos modelos de machine learning. Além disso, incluímos os códigos dos modelos escolhidos para o processamento de linguagem natural. Os modelos são:

1. Naive Bayes
2. Random Forest
3. SVM
4. XGBoost
5. Embedding Layers

A análise detalhada de cada um dos modelos pode ser acessada na documentação, [neste link](https://github.com/Inteli-College/2024-1B-T10-SI06-G03/blob/main/docs/documentacao.md#c7.2).

### Criação do Dataset de testagem dos Modelos:

In [ ]:
def merge_sentiment_column(classification_csv, word2vec_csv, output_csv):
    """
    Carrega dois arquivos CSV, adiciona a coluna 'sentiment' do primeiro arquivo
    ao segundo arquivo, e salva o resultado em um novo arquivo CSV.

    Args:
    classification_csv (str): Caminho para o arquivo CSV contendo a coluna 'sentiment'.
    word2vec_csv (str): Caminho para o arquivo CSV contendo os vetores de palavras.
    output_csv (str): Caminho para o arquivo CSV onde o resultado será salvo.
    
    Returns:
    None
    """
    # Carregar os arquivos CSV
    classification_text_data = pd.read_csv(classification_csv)
    word_2_vec_data = pd.read_csv(word2vec_csv)

    # Selecionar a coluna 'sentiment' do arquivo classification_text_data
    sentiment_column = classification_text_data['sentiment']

    # Adicionar a coluna 'sentiment' no arquivo word_2_vec_data na primeira coluna (coluna 0)
    word_2_vec_data.insert(0, 'sentiment', sentiment_column)

    # Verificar o resultado
    print(word_2_vec_data.head())

    # Salvar o novo DataFrame em um novo arquivo CSV
    word_2_vec_data.to_csv(output_csv, index=False)

In [ ]:
# Chamando a função que 1: vetoriza o banco de dados utilizando o Word2Vec e 2: cria o dataset para treinar os modelos 
merge_sentiment_column('./data/processed_text_data.csv', './data/word_2_vec_text_data_300.csv', './data/data_for_training_models.csv')

## Naive Bayes

Nesta seção, utilizaremos o modelo Gaussian Naive Bayes para classificar as frases como positivas ou negativas.

<b> Diferença entre `MultiNomialNB` e `GaussianNB` </b>
 Como nossos dados são contínuos e possuem valores negativos, eles acabam sendo inapropriados para uso direto com `MultinomialNB`. 
 
 Para esses tipos de dados, modelos como o GaussianNB, que presume que as características seguem uma distribuição normal, são mais adequados.

 Vale ressaltar que o `MultiNomialNB` é utilizada para:

 - Dados Discretos: Sendo ideal para dados que representam contagens ou frequências de eventos. Por exemplo, é comum em tarefas de classificação de texto onde os recursos são normalmente as contagens ou frequências de termos (bag of words).

- Dados Positivos: Ele requer que os recursos sejam não-negativos, pois representa a frequência de ocorrência de características. Em contextos onde os recursos podem ser negativos, como é o caso em muitos conjuntos de dados de características contínuas ou normalizadas, o MultinomialNB não seria adequado sem transformações prévias que garantam a não-negatividade.

Para ver o resultado obtido, basta rodar o código abaixo.

In [ ]:
from sklearn.metrics import classification_report # não modificar o local dessa importação
import pickle

def train_and_save_model(file_path, model_save_path):
    """
    Treina e salva um modelo de classificação Naive Bayes em um arquivo.

    Input:
    file_path (str): O caminho para o arquivo CSV contendo os dados de treinamento.
    model_save_path (str): O caminho para salvar o arquivo do modelo treinado.

    Output:
    dict: Um dicionário contendo a acurácia do modelo, o relatório de classificação, e os conjuntos de teste.
    """
    # Lendo os dados do arquivo CSV
    data = pd.read_csv(file_path)

    # Dividindo os dados em conjuntos de treino e teste
    X = data.drop('sentiment', axis=1)
    y = data['sentiment']
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

    # Modelo de classificação
    model = GaussianNB()

    # Treinando o modelo
    model.fit(X_train, y_train)

    # Salvando o modelo treinado com pickle
    with open(model_save_path, 'wb') as model_file:
        pickle.dump(model, model_file)

    # Predições no conjunto de teste
    y_pred = model.predict(X_test)

    # Avaliação do modelo
    accuracy = accuracy_score(y_test, y_pred)
    class_report = classification_report(y_test, y_pred, output_dict=True)

    return {
        "accuracy": accuracy,
        "classification_report": class_report,
        "X_test": X_test,  # Adiciona X_test ao retorno para uso posterior
        "y_test": y_test   # Adiciona y_test ao retorno para uso posterior
    }

### Salvamento do Modelo

In [ ]:
def load_and_predict(model_load_path, X_test):
    """
    Carrega um modelo salvo e faz previsões no conjunto de teste.

    Input:
    model_load_path (str): O caminho para o arquivo do modelo salvo.
    X_test (DataFrame): Conjunto de dados de teste.

    Output:
    array: Predições feitas pelo modelo carregado.
    """
    # Carregar o modelo salvo com pickle
    with open(model_load_path, 'rb') as model_file:
        loaded_model = pickle.load(model_file)

    # Fazer previsões usando o modelo carregado
    return loaded_model.predict(X_test)

# Treinando e salvando o modelo
result = train_and_save_model('./data/data_for_training_models.csv', './models/naive_bayes_model.pkl')
print("Acurácia:", result["accuracy"])
print("Relatório de Classificação:\n", result["classification_report"])

# Carregar o modelo salvo e fazer previsões
y_pred_loaded_model = load_and_predict('./models/naive_bayes_model.pkl', result["X_test"])

# Avaliação das previsões do modelo carregado
accuracy_loaded_model = accuracy_score(result["y_test"], y_pred_loaded_model)
class_report_loaded_model = classification_report(result["y_test"], y_pred_loaded_model)

print("Acurácia do Modelo Carregado:", accuracy_loaded_model)
print("Relatório de Classificação do Modelo Carregado:\n", class_report_loaded_model)

## Random Forest

O modelo Random Forest é uma técnica de aprendizado de máquina utilizada em problemas de classificação e regressão. É uma extensão do método de Árvores de Decisão, uma abordagem de aprendizado supervisionado que cria uma árvore de decisão com base nas características dos dados de treinamento. No entanto, em vez de depender de uma única árvore de decisão, o modelo Random Forest constrói uma "floresta" de árvores de decisão e combina suas previsões para obter uma previsão final.

A principal ideia por trás do modelo Random Forest é criar uma variedade de árvores de decisão durante o treinamento, cada uma delas sendo treinada com uma amostra aleatória dos dados de treinamento e com um conjunto aleatório de características. Isso é conhecido como bootstrap aggregating, ou bagging, e ajuda a reduzir a variância do modelo, melhorando sua capacidade de generalização. 

Além disso, durante a divisão de cada nó em uma árvore de decisão, o modelo Random Forest seleciona um subconjunto aleatório das características, em vez de usar todas elas. Isso ajuda a reduzir a correlação entre as árvores individuais na floresta, tornando o modelo mais robusto a overfitting e melhorando sua capacidade de generalização.

A seguir, o modelo Random Forest será aplicado para classificar o sentimento com base nos comentários fornecidos.

#### Utilizando a base de dados do Word2Vec

In [ ]:
def train_random_forest_model_W2V(data_path):
    """
    Treina um modelo Random Forest para classificação de sentimentos.

    Inputs: data_path (str): O caminho para o arquivo CSV contendo os dados.

    Outputs: Métricas do Modelo Random Forest Utilizando a base de dados do Word2Vec.
    """

    # Carregar os dados
    data = pd.read_csv(data_path)
    
    # Dividir os dados em features (X) e target (y)
    X = data.drop('sentiment', axis=1)
    y = data['sentiment']

    # Dividir os dados em conjuntos de treinamento e teste
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    # Criar o classificador Random Forest com ajuste de pesos das classes
    rf_classifier = RandomForestClassifier(class_weight='balanced')

    # Treinar o classificador com os dados de treinamento
    rf_classifier.fit(X_train, y_train)

    # Fazer previsões nos dados de teste
    y_pred = rf_classifier.predict(X_test)

    # Converter os valores das classes para strings
    target_names = [str(cls) for cls in rf_classifier.classes_]

    # Exibir o relatório de classificação para cada classe
    class_report = classification_report(y_test, y_pred, target_names=target_names)
    print("Classification Report:\n", class_report)
    
    # Calcular a matriz de confusão
    conf_matrix = confusion_matrix(y_test, y_pred)
    
    # Visualizar a matriz de confusão usando seaborn
    plt.figure(figsize=(10, 8))
    sns.heatmap(conf_matrix, annot=True, fmt="d", cmap="Blues", xticklabels=target_names, yticklabels=target_names)
    plt.title("Matriz de Confusão")
    plt.xlabel("Valor Previsto")
    plt.ylabel("Valor Real")
    plt.show()


In [ ]:
# Chamar a função com o caminho para o arquivo CSV
train_random_forest_model_W2V('./data/data_for_training_models.csv')

## SVM

O modelo SVM *(Support Vector Machine)* é uma técnica de aprendizado de máquina utilizada tanto para tarefas de classificação quanto de regressão. Ele opera identificando o melhor hiperplano que separa os dados de diferentes classes no espaço de características.

A ideia central do SVM é maximizar a margem de separação entre as classes de dados. Esta margem é definida como a distância entre o hiperplano de separação e os pontos de dados mais próximos, conhecidos como vetores de suporte. Ao maximizar essa margem, o modelo SVM busca melhorar sua capacidade de generalização, reduzindo a probabilidade de overfitting.

O SVM é particularmente poderoso em espaços de alta dimensão e é eficaz mesmo quando o número de dimensões é maior do que o número de amostras. Além disso, quando os dados não são linearmente separáveis, o SVM utiliza uma técnica chamada *kernel trick*. Esta técnica transforma os dados para um espaço de maior dimensão onde é possível encontrar um hiperplano que separe as classes. Os kernels mais comuns utilizados são o linear, o polinomial e o RBF (*Radial Basis Function*).

A seguir, o grupo *Thunder* aplicará o modelo SVM à tarefa de classificação de sentimentos com base nos comentários fornecidos.

In [ ]:
from sklearn.metrics import classification_report # não modificar o local dessa importação

def svm_model(csv_word2vec):
    """
    Treina e avalia um modelo SVM usando um conjunto de dados fornecido.

    Inputs: csv_word2vec (pd.DataFrame): DataFrame contendo os dados de entrada, com a coluna 'sentiment' como a variável alvo.

    Outputs: None
    
    """

    # Separando características (X) e rótulos (y)
    X = csv_word2vec.drop(columns=["sentiment"])
    y = csv_word2vec["sentiment"]
    
    # Dividindo o conjunto de dados em conjuntos de treino e teste
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

    # Padronizar os dados
    scaler = StandardScaler()
    X_train = scaler.fit_transform(X_train)
    X_test = scaler.transform(X_test)

    # Treinando o modelo SVM com kernel RBF
    svm_model = SVC(kernel="rbf", probability=True)
    svm_model.fit(X_train, y_train)

    # Fazendo previsões no conjunto de teste
    y_pred = svm_model.predict(X_test)

    # Calculando a acurácia
    accuracy = accuracy_score(y_test, y_pred)
    print("Accuracy:", accuracy)

    # Calculando a matriz de confusão
    conf_matrix = confusion_matrix(y_test, y_pred)
    print("Confusion Matrix:\n", conf_matrix)

    # Exibindo o relatório de classificação
    class_report = classification_report(y_test, y_pred)
    print("Classification Report:\n", class_report)

    # Binarizando as classes para cálculo da curva ROC e AUC
    y_test_binarized = label_binarize(y_test, classes=np.unique(y))
    n_classes = y_test_binarized.shape[1]

    # Calculando as probabilidades para cada classe
    y_pred_prob = svm_model.predict_proba(X_test)

    # Calculando a curva ROC e a área sob a curva (AUC) para cada classe
    fpr = dict()
    tpr = dict()
    roc_auc = dict()
    for i in range(n_classes):
        fpr[i], tpr[i], _ = roc_curve(y_test_binarized[:, i], y_pred_prob[:, i])
        roc_auc[i] = roc_auc_score(y_test_binarized[:, i], y_pred_prob[:, i])

    # Plotando as curvas ROC
    plt.figure()
    colors = cycle(['aqua', 'darkorange', 'cornflowerblue'])
    for i, color in zip(range(n_classes), colors):
        plt.plot(fpr[i], tpr[i], color=color, lw=2,
                 label='ROC curve of class {0} (area = {1:0.2f})'
                 ''.format(i, roc_auc[i]))
    plt.plot([0, 1], [0, 1], 'k--', lw=2)
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('Receiver Operating Characteristic (ROC)')
    plt.legend(loc="lower right")
    plt.show()

    # Exibindo os scores AUC
    for i in range(n_classes):
        print(f"ROC AUC Score for class {i}: {roc_auc[i]}")


In [ ]:
# Lendo os dados do arquivo CSV
data = pd.read_csv("./data/data_for_training_models.csv")
svm_model(data)

### Criando dados ficticios para uso em testes isolados

In [ ]:
from sklearn.datasets import make_classification
def create_sample_data():
    """
    Cria um conjunto de dados de exemplo para testar a função svm_model.

    Returns:
    data (pd.DataFrame): DataFrame contendo os dados de exemplo.
    """
    X, y = make_classification(n_samples=1000, n_features=20, n_classes=3, n_informative=3, n_clusters_per_class=1, random_state=42)
    data = pd.DataFrame(X, columns=[f'feature_{i}' for i in range(X.shape[1])])
    data['sentiment'] = y
    return data


### Teste de implantação do modelo


In [ ]:
data = create_sample_data()
svm_model(data)

## XGBooster

O modelo XGBoost, ou *Extreme Gradient Boosting*, é uma técnica avançada de aprendizado de máquina que combina previsões de múltiplos modelos fracos para formar um modelo robusto e preciso. Amplamente utilizado em competições de ciência de dados, o XGBoost é conhecido por seu desempenho superior, eficiência computacional e capacidade de lidar com grandes volumes de dados. Suas características, como regularização L1 e L2, manuseio automático de dados faltantes, paralelização e flexibilidade em funções de perda, tornam-no uma escolha ideal para tarefas de classificação e regressão.

Neste projeto, o XGBoost foi aplicado para classificar sentimentos em três categorias: negativo (-1), neutro (0) e positivo (1), utilizando um conjunto de dados com vetores *Word2Vec*. O objetivo foi construir um modelo capaz de prever com precisão o sentimento expressado em textos, avaliando seu desempenho por meio de métricas como acurácia, precisão, recall e F1-score, visualizando os resultados em uma matriz de confusão.

In [ ]:
import pandas as pd
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns

def train_xgboost_model(file_path):
    """
    Treina um modelo XGBoost utilizando Word2Vec e exibe as métricas de avaliação.

    Inputs: file_path (str): Caminho para o arquivo CSV contendo os dados de treinamento.

    Outputs: dict: Um dicionário contendo as métricas de avaliação do modelo ou None se ocorrer um erro.
    """
    try:
        # Carregar os dados
        data = pd.read_csv(file_path)

        # Verificar a presença da coluna 'sentiment'
        if 'sentiment' not in data.columns:
            raise KeyError("'sentiment' não encontrado nas colunas do DataFrame")

        # Dividir os dados em features (X) e target (y)
        X = data.drop('sentiment', axis=1)
        y = data['sentiment']

        # Re-rotular os rótulos para garantir que estão no intervalo [0, num_class)
        y = LabelEncoder().fit_transform(y)

        # Dividir os dados em conjuntos de treinamento e teste
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

        # Criar o DMatrix para o XGBoost
        dtrain = xgb.DMatrix(X_train, label=y_train)
        dtest = xgb.DMatrix(X_test, label=y_test)

        # Definir os parâmetros do XGBoost
        params = {
            'max_depth': 6,
            'eta': 0.3,
            'objective': 'multi:softmax',
            'num_class': 3
        }

        # Treinar o modelo
        bst = xgb.train(params, dtrain, num_boost_round=100)

        # Fazer previsões
        y_pred = bst.predict(dtest)

        # Exibindo o relatório de classificação
        class_report = classification_report(y_test, y_pred)
        print("Classification Report:\n", class_report)

        # Calcular e visualizar a matriz de confusão
        conf_matrix = confusion_matrix(y_test, y_pred)
        plt.figure(figsize=(10, 8))
        sns.heatmap(conf_matrix, annot=True, fmt="d", cmap="Blues", xticklabels=['Negativo', 'Neutro', 'Positivo'], yticklabels=['Negativo', 'Neutro', 'Positivo'])
        plt.title("Matriz de Confusão")
        plt.xlabel("Valor Previsto")
        plt.ylabel("Valor Real")
        plt.show()

        return metrics

    except Exception as e:
        print(f"Ocorreu um erro: {e}")
        return None

In [ ]:
# Chamar a função com o caminho para o arquivo CSV
metrics = train_xgboost_model('./data/data_for_training_models.csv')

## Embedding Layers

In [ ]:
def embedding_layers_model_encoding(csv_file_path):
    """
    Treina um modelo LSTM com dados de vetores de palavras e calcula métricas de avaliação.

    Inputs: csv_file_path (str): Caminho para o arquivo CSV contendo os dados.
    
    Outputs: dict: Um dicionário contendo as métricas de avaliação (accuracy, precision, recall, f1).
    """
    # Carregar o arquivo CSV
    data = pd.read_csv(csv_file_path)

    # Combinar as colunas de vetores em uma única coluna 'vectors'
    vector_columns = [str(i) for i in range(300)]
    data['vectors'] = data[vector_columns].values.tolist()

    # Remover as colunas de vetores separadas
    data = data.drop(columns=vector_columns)

    # Converter a coluna 'vectors' de listas de listas para arrays numpy
    data['vectors'] = data['vectors'].apply(np.array)

    # Criar as sequências de vetores
    X = np.array(data['vectors'].tolist())
    y = np.array(data['sentiment'])  # Usando a coluna 'sentiment' como rótulo

    # Expandir as dimensões de X para ter a forma correta para LSTM
    X = np.expand_dims(X, axis=-1)

    # One-hot encode os rótulos
    onehot_encoder = OneHotEncoder(sparse_output=False)
    y = onehot_encoder.fit_transform(y.reshape(-1, 1))

    # Dividir os dados em conjuntos de treino e teste
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    # Definir e compilar o modelo
    model = Sequential()
    model.add(LSTM(128, input_shape=(300, 1), return_sequences=False))
    model.add(Dense(3, activation='softmax'))

    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

    model.summary()

    # Treinar o modelo
    model.fit(X_train, y_train, epochs=50, batch_size=32, validation_split=0.2)

    # Prever os resultados
    y_pred = model.predict(X_test)
    y_pred_classes = np.argmax(y_pred, axis=1)
    y_test_classes = np.argmax(y_test, axis=1)

    # Calcular as métricas
    accuracy = accuracy_score(y_test_classes, y_pred_classes)
    precision = precision_score(y_test_classes, y_pred_classes, average='macro')  # 'macro' para multiclasse
    recall = recall_score(y_test_classes, y_pred_classes, average='macro')        # 'macro' para multiclasse
    f1 = f1_score(y_test_classes, y_pred_classes, average='macro')                # 'macro' para multiclasse

    # Imprimir a matriz de avaliação
    print(f"Accuracy: {accuracy:.4f}")
    print(f"Precision: {precision:.4f}")
    print(f"Recall: {recall:.4f}")
    print(f"F1 Score: {f1:.4f}")

    # Calcular e imprimir o relatório de classificação
    class_report = classification_report(y_test_classes, y_pred_classes, target_names=['Negativo', 'Neutro', 'Positivo'])
    print("\nRelatório de Classificação:\n", class_report)

    # Calcular e visualizar a matriz de confusão
    conf_matrix = confusion_matrix(y_test_classes, y_pred_classes)
    plt.figure(figsize=(10, 8))
    sns.heatmap(conf_matrix, annot=True, fmt="d", cmap="Blues", xticklabels=['Negativo', 'Neutro', 'Positivo'], yticklabels=['Negativo', 'Neutro', 'Positivo'])
    plt.title("Matriz de Confusão")
    plt.xlabel("Valor Previsto")
    plt.ylabel("Valor Real")
    plt.show()

    metrics = {
        'accuracy': accuracy,
        'precision': precision,
        'recall': recall,
        'f1_score': f1,
        'classification_report': class_report
    }

    return metrics

In [ ]:
# Chamando a função com o banco de dados que está sendo utilizado
embedding_layers_model_encoding('./data/data_for_training_models.csv')

# Referências
Segue as referências utilizadas durante a execução deste Documento Jupyther.

1.  Métricas de Avaliação: Acesso em: 19 de Maio de 2024. Disponível em: https://vitorborbarodrigues.medium.com/m%C3%A9tricas-de-avalia%C3%A7%C3%A3o-acur%C3%A1cia-precis%C3%A3o-recall-quais-as-diferen%C3%A7as-c8f05e0a513c
2. Breiman, L. (2001). Random Forests. Machine Learning, 45(1), 5-32. Acesso em: 21 de Maio de 2024. Disponível em: https://link.springer.com/article/10.1023/A:1010933404324.
3. Cortes, C., & Vapnik, V. (1995). Support-vector networks. Machine Learning, 20(3), 273-297. Acesso em: 21 de Maio de 2024. Disponível em: https://link.springer.com/article/10.1007/BF00994018.
4. Chen, T., & Guestrin, C. (2016). XGBoost: A Scalable Tree Boosting System. Proceedings of the 22nd ACM SIGKDD International Conference on Knowledge Discovery and Data Mining, 785-794. Acesso em: 21 de Maio de 2024. Disponível em: https://dl.acm.org/doi/10.1145/2939672.2939785.
5. Google Cloud Tech. "Getting started with keras". Acesso em 19 de maio de 2024. https://www.youtube.com/watch?v=J6Ok8p463C4
6. Código Fonte TV. "Tensor flower". Acesso em: 22 de maio de 2024. Disponível em: https://www.youtube.com/watch?v=2eYLt1NA4Ss
7. ChatGPT 3.5. OpenAI, utilizado em várias interações e consultas durante o desenvolvimento do projeto. Acesso em: 13 maio 2024.
8. ChatGPT 4. OpenAI, utilizado em várias interações e consultas durante o desenvolvimento do projeto. Acesso em: 13 maio 2024.
9. Understanding Word Vectors. Acessado em: 23 de maio de 2024. Disponível em: https://gist.github.com/aparrish/2f562e3737544cf29aaf1af30362f469
10. Codebasics. "Whats is word 2 vec". Acessado em 17 de maio de 2024. Diwponível em: https://www.youtube.com/watch?v=hQwFeIupNP0
11. Entenda como funciona o Random Foresr. Acessado em : 20 de maio de 2024. Disponível em: https://didatica.tech/o-que-e-e-como-funciona-o-algoritmo-randomforest/
12. Word2Vec: interpretação da linguagem humana com Word embedding. Acessado em 21 de maio de 2024. Disponível em: https://www.alura.com.br/conteudo/introducao-word-embedding
13. SVM. Acessado em 22 de maio de 2024. Disponível em: https://www.inf.ufpr.br/dagoncalves/IA07.pdf
14. Acessado em 24 de maio de 2024. Disponível em: https://medium.com/data-hackers/entendendo-o-que-%C3%A9-matriz-de-confus%C3%A3o-com-python-114e683ec509